<a href="https://colab.research.google.com/github/jasur-rasulov/New-York-City-Taxi-Fare-Forecasting/blob/master/Copy_of_IDS561_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## *Loading the Train and Test Datasets*

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
% matplotlib inline
plt.style.use('seaborn-whitegrid')
import sklearn

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train = pd.read_csv("/content/drive/Team Drives/IDS 561 - Big Data Group Project/ny_fare_train.csv")
test = pd.read_csv("/content/drive/Team Drives/IDS 561 - Big Data Group Project/ny_fare_test.csv")

In [0]:
print(train.shape, test.shape)

(199999, 8) (54174, 8)


In [0]:
train.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,26:21.0,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,52:16.0,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,35:00.0,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,30:42.0,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,51:00.0,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [0]:
test.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,14:53.0,5.5,2014-03-03 11:14:53 UTC,-73.980647,40.721606,-73.994044,40.727628,1
1,35:54.0,6.0,2015-03-14 13:35:54 UTC,-73.958839,40.815304,-73.966675,40.804443,4
2,43:51.0,8.1,2010-05-24 23:43:51 UTC,-73.985327,40.760201,-73.981010,40.744734,1
3,53:08.0,8.5,2015-03-09 16:53:08 UTC,-73.982170,40.768639,-73.992203,40.743530,1
4,15:00.0,6.9,2011-11-25 22:15:00 UTC,-74.005458,40.741027,-74.004225,40.720920,1


## ***Checking Data Types***
---

In [0]:
train.dtypes

key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

##**Examining the number of missing values for each column on both datasets**
---

In [0]:
print(train.isnull().sum())

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    1
dropoff_latitude     1
passenger_count      0
dtype: int64


In [0]:
print(test.isnull().sum())

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    2
dropoff_latitude     2
passenger_count      0
dtype: int64


## ***Deleting Missing Values***

---

In [0]:
train = train.dropna(how = 'any', axis = 'rows')
test = test.dropna(how = 'any', axis = 'rows')

## ***Data Engineering***

---

### ***Creating a function to represent the absolute difference (Manhattan distance) between the pickup and dropoff locations***

---

In [0]:
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

add_travel_vector_features(train)

### ***Converting the Values of Pickup_Datetime to Year, Month, Date, Day of Week, and Hour Columns***

---

In [0]:
train['pickup_datetime']  = pd.to_datetime(train['pickup_datetime'])
test['pickup_datetime']  = pd.to_datetime(test['pickup_datetime'])
data = [train,test]
for i in data:
    i['Year'] = i['pickup_datetime'].dt.year
    i['Month'] = i['pickup_datetime'].dt.month
    i['Date'] = i['pickup_datetime'].dt.day
    i['Day of Week'] = i['pickup_datetime'].dt.dayofweek
    i['Hour'] = i['pickup_datetime'].dt.hour

In [0]:
train.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,abs_diff_longitude,abs_diff_latitude,Year,Month,Date,Day of Week,Hour
0,26:21.0,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1,0.002701,0.009041,2009,6,15,0,17
1,52:16.0,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1,0.036780,0.070701,2010,1,5,1,16
2,35:00.0,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2,0.008504,0.010708,2011,8,18,3,0
3,30:42.0,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1,0.004437,0.024949,2012,4,21,5,4
4,51:00.0,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1,0.011440,0.015754,2010,3,9,1,7
